In [1]:
from torchsummary import summary
from python_lib.modules import *
import torch
import numpy as np
import os

c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chiny\anaconda3\envs\pytorch\lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html
SpeechBrain could not find any working torchaudio backend. Audio files may fail to load. Follow this link for instructions and troubleshooting: https://pytorch.org/audio/stable/index.html


In [10]:
model = ECAPA_TDNN(input_size = 2, channels=[8,8,8,8,16], lin_neurons=6, device = "cuda").to("cuda")
input_feats = torch.rand([10,2,64]).to("cuda")
output = model(input_feats)
output.shape

torch.Size([10, 6])

In [12]:
# Default Activation fn = Relu and group = 1

initialblock = TDNNBlock(in_channels=2,
                        out_channels=8,
                        kernel_size=5,
                        dilation= 1).to("cuda")

x0 = initialblock(input_feats)
print("Initial out: ",x0.shape)

seres2_1 = SERes2NetBlock(in_channels=8,
                          out_channels=8, 
                          res2net_scale=8, 
                          se_channels=128, 
                          kernel_size=3, 
                          dilation=2).to("cuda")

x1 = seres2_1(x0)
print("SERes2_1 out: ",x1.shape)

seres2_2 = SERes2NetBlock(in_channels=8,
                          out_channels=8, 
                          res2net_scale=8, 
                          se_channels=128, 
                          kernel_size=3, 
                          dilation=3).to("cuda")

x2 = seres2_2(x1)
print("SERes2_2 out: ",x2.shape)

seres2_3 = SERes2NetBlock(in_channels=8,
                          out_channels=8, 
                          res2net_scale=8, 
                          se_channels=128, 
                          kernel_size=3, 
                          dilation=4).to("cuda")

x3 = seres2_3(x2)
print("SERes2_3 out: ",x3.shape)

mfa = TDNNBlock(in_channels=(8 * 3),
                out_channels=16,
                kernel_size=1,
                dilation= 1).to("cuda")

x4 = mfa(torch.cat([x1,x2,x3], dim=1))
print("mfa out: ",x3.shape)

asp = AttentiveStatisticsPooling(16, 128, True).to("cuda")
x5 = asp(x4)
x6 = nn.BatchNorm1d(16 * 2).to("cuda")(x5)
print("asp out: ",x6.shape)

# Remember to x2 to account for the mean & std in asp
final = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=16 * 2, out_features=6),
        ).to("cuda")

y = final(x6)
print("final out: ",y.shape)

Initial out:  torch.Size([10, 8, 64])
SERes2_1 out:  torch.Size([10, 8, 64])
SERes2_2 out:  torch.Size([10, 8, 64])
SERes2_3 out:  torch.Size([10, 8, 64])
mfa out:  torch.Size([10, 8, 64])
asp out:  torch.Size([10, 32, 1])
final out:  torch.Size([10, 6])


## Disecting SERes2NetBlock

Since in_channel == out_channel, shortcut is not necessary

In [13]:
tdnn1 = TDNNBlock(in_channels=8, 
                  out_channels=8, 
                  kernel_size=1, 
                  dilation=1).to("cuda")

z0 = tdnn1(x0)
print(z0.shape)
res2net = Res2NetBlock(in_channels=8, 
                       out_channels=8, 
                       scale=8, 
                       kernel_size=3, 
                       dilation=1).to("cuda")

z1 = res2net(z0)
print(z1.shape)

tdnn2 = TDNNBlock(in_channels=8, 
                  out_channels=8, 
                  kernel_size=1, 
                  dilation=1).to("cuda")

z2 = tdnn2(z1)
print(z2.shape)

se_block = SEBlock(in_channels=8, 
                   se_channels=128, 
                   out_channels=8).to("cuda")

z3 = se_block(z2)
print((z3 + x0).shape)


torch.Size([10, 8, 64])
torch.Size([10, 8, 64])
torch.Size([10, 8, 64])
torch.Size([10, 8, 64])


## Disecting Res2NetBlock

Here Scale = 8

In [17]:
scale = 8
in_channel = 8//scale # 1
hidden_channel = 8//scale # 1

block1 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block2 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block3 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block4 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block5 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block6 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")
block7 = TDNNBlock(in_channels=1, 
                  out_channels=1, 
                  kernel_size=3, 
                  dilation=1).to("cuda")

chunks = torch.chunk(z0, 8, 1)
# for i in chunks:
#     print(i.shape)

b7 = block7(chunks[0])
print(chunks[0].shape)
print(b7.shape)

torch.Size([10, 1, 64])
torch.Size([10, 1, 64])


In [19]:
from speechbrain.nnet.CNN import Conv1d as _Conv1d

class Conv1d(_Conv1d):
    """1D convolution. Skip transpose is used to improve efficiency."""

    def __init__(self, *args, **kwargs):
        super().__init__(skip_transpose=True, *args, **kwargs)

c1 = Conv1d(in_channels=1,
            out_channels=1,
            kernel_size=3,
            dilation=1,
            groups=1).to("cuda")

c2 = nn.Conv1d(1,1,3,1,0,1).to("cuda")

print(c1(chunks[0]).shape)
print(c2(chunks[0]).shape)

torch.Size([10, 1, 64])
torch.Size([10, 1, 62])
